In [64]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random                    #random song recommandation
from Levenshtein import ratio
from config import client_id, client_secret
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

<h2> Dataframe Top_100 </h2>

In [3]:
# 1. get top 100 and create Dataframe with artists & titles

In [5]:
url = "https://www.billboard.com/charts/hot-100/"

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print(f"Error: {response.status_code}")

In [6]:
#artists
soup.find_all("span", class_="c-label")

[<span class="c-label a-font-primary-bold-xxl lrv-u-padding-lr-1 lrv-u-padding-t-025">
 	
 	1
 </span>,
 <span class="c-label a-font-primary-bold-xxl lrv-u-padding-lr-1 lrv-u-padding-t-025">
 	
 	17
 </span>,
 <span class="c-label a-font-primary-bold-xxl lrv-u-padding-lr-1 lrv-u-padding-t-025">
 	
 	30
 </span>,
 <span class="c-label a-font-primary-bold-l u-font-size-32@tablet u-letter-spacing-0080@tablet">
 	
 	1
 </span>,
 <span class="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet">
 	
 	Shaboozey
 </span>,
 <span class="c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb-050@mobile-max u-font-size-32@tablet">
 	
 	1
 </span>,
 <span class="c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb

In [9]:
artists=[]
for a in soup.select("span.c-label.a-no-trucate"):
    artists.append(a.get_text(strip=True))
artists

['Shaboozey',
 'Lady Gaga & Bruno Mars',
 'Billie Eilish',
 'Sabrina Carpenter',
 'Teddy Swims',
 'Post Malone Featuring Morgan Wallen',
 'Sabrina Carpenter',
 'Benson Boone',
 'Morgan Wallen',
 'Tyler, The Creator Featuring GloRilla, Sexyy Red & Lil Wayne',
 'Chappell Roan',
 'Hozier',
 'Gracie Abrams',
 'Tyler, The Creator Featuring Daniel Caesar',
 'ROSE & Bruno Mars',
 'Sabrina Carpenter',
 'Kendrick Lamar',
 'Jelly Roll',
 'The Weeknd & Playboi Carti',
 'Chappell Roan',
 'Tyler, The Creator Featuring Teezo Touchdown',
 'Morgan Wallen',
 'Oscar Maydon & Fuerza Regida',
 'Myles Smith',
 'Tyler, The Creator',
 'Gigi Perez',
 'Tommy Richman',
 'Gracie Abrams',
 'Tyler, The Creator Featuring Lola Young',
 'GloRilla & Sexyy Red',
 'Marshmello & Kane Brown',
 'Billie Eilish',
 'Morgan Wallen Featuring ERNEST',
 'Sabrina Carpenter',
 'Teddy Swims',
 'GloRilla',
 'Tyler, The Creator',
 'Zach Bryan',
 'Tyler, The Creator Featuring ScHoolboy Q & Santigold',
 'Ella Langley Featuring Riley Gre

In [11]:
#titles
soup.find_all(class_ = 'o-chart-results-list-row-container')                # find all containers

[<div class="o-chart-results-list-row-container">
 <ul class="o-chart-results-list-row // lrv-a-unstyle-list lrv-u-flex u-height-200 u-height-100@mobile-max u-height-100@tablet-only lrv-u-background-color-white a-chart-has-chart-detail" data-ajax="" data-detail-target="1">
 <li class="o-chart-results-list__item // lrv-u-background-color-black lrv-u-color-white u-width-100 u-width-55@mobile-max u-width-55@tablet-only lrv-u-height-100p lrv-u-flex lrv-u-flex-direction-column lrv-u-flex-shrink-0 lrv-u-align-items-center lrv-u-justify-content-center lrv-u-border-b-1 u-border-b-0@mobile-max lrv-u-border-color-grey">
 <span class="c-label a-font-primary-bold-l u-font-size-32@tablet u-letter-spacing-0080@tablet">
 	
 	1
 </span>
 <div class="c-svg u-height-10@mobile-max u-width-10@mobile-max u-hidden@tablet">
 <svg height="10.157" width="10.157" xmlns="http://www.w3.org/2000/svg"><path d="M7.727 5.732H0V4.426h7.727L4.322.923 5.219 0l4.938 5.079-4.938 5.079-.9-.923z" data-name="Arrow" fill="#82

In [13]:
titles = []
for title in soup.find_all(class_ = 'o-chart-results-list-row-container'):     #above h3
   titles.append(title.h3.get_text(strip = True))
titles

['A Bar Song (Tipsy)',
 'Die With A Smile',
 'Birds Of A Feather',
 'Espresso',
 'Lose Control',
 'I Had Some Help',
 'Taste',
 'Beautiful Things',
 'Love Somebody',
 'Sticky',
 'Good Luck, Babe!',
 'Too Sweet',
 "That's So True",
 'St. Chroma',
 'APT.',
 'Please Please Please',
 'Not Like Us',
 'I Am Not Okay',
 'Timeless',
 'Hot To Go!',
 'Darling, I',
 'Lies Lies Lies',
 'Tu Boda',
 'Stargazing',
 'Rah Tah Tah',
 'Sailor Song',
 'Million Dollar Baby',
 "I Love You, I'm Sorry",
 'Like Him',
 'Whatchu Kno About Me',
 'Miles On It',
 'Wildflower',
 'Cowgirls',
 'Bed Chem',
 'The Door',
 'TGIF',
 'Noid',
 'Pink Skies',
 'Thought I Was Dead',
 'You Look Like You Love Me',
 "Ain't No Love In Oklahoma",
 'Austin',
 'I Can Do It With A Broken Heart',
 'Who',
 'High Road',
 'Pink Pony Club',
 "Wind Up Missin' You",
 'Chill Bae',
 'Kehlani',
 'Pour Me A Drink',
 'Dos Dias',
 'Judge Judy',
 'Dancing In The Flames',
 'Light Year (Practice)',
 '25',
 'Help Me',
 "It's Ok I'm Ok",
 'Disease',
 'T

In [15]:
# create Dataframe 

df= pd.DataFrame({
    "Titles": titles,
    "Artist": artists})
df

,Titles,Artist
0,A Bar Song (Tipsy),Shaboozey
1,Die With A Smile,Lady Gaga & Bruno Mars
2,Birds Of A Feather,Billie Eilish
3,Espresso,Sabrina Carpenter
4,Lose Control,Teddy Swims
...,...,...
95,Hollon,GloRilla
96,Lonely Road,mgk & Jelly Roll
97,Change Me,BigXthaPlug
98,Him All Along,Gunna


In [17]:
#df.to_csv("df_hot_100.csv", index=False)

In [19]:
# Create a Python program that takes a user input, checks if it is present in your scraped DataFrame

In [37]:
print("You can type 'exit' at any time to quit.")
while True:
    user_input = input("Please enter a song: ").strip()
    # Exit Condition!
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    if user_input in df["Titles"].values:
        random_song = df.sample(1)["Titles"].iloc[0]                                  #randomly one row
        print(f"Your song is popular! Here's another suggestion: '{random_song}'")
    else:
        print("Sorry, your song is not hot enough.")

You can type 'exit' at any time to quit.


Please enter a song:  exit


Goodbye!


In [ ]:
# Handling Typos
#!pip install python-Levenshtein
"""The Levenshtein algorithm measures how many operations (insertion, deletion, or substitution) are required to transform one string into another.
The result is a similarity score ranging from 0 (no similarity) to 1 (identical). If the similarity of a title exceeds the current best_score, that title is stored as the new best_match."""

In [86]:
print("You can type 'exit' at any time to quit.")
while True:
    user_input = input("Please enter a song: ").strip()
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
                                                                # Levenshtein
    best_match = None
    best_score = 0
    
    for title in df["Titles"]:
        similarity = ratio(user_input, title)       # calculates similarity between 0 and 1
        if similarity > best_score:
            best_score = similarity
            best_match = title

    # check if similarity is high enough 
    if best_score >= 0.99:
        random_song = df.sample(1)["Titles"].iloc[0]
        print(f"It's hot! Here's is another suggestion: '{random_song}'")
    elif best_score >= 0.6:                                     # similarity treshhold: 60 % ?
        random_song = df.sample(1)["Titles"].iloc[0]  
        print(f"Did you mean '{best_match}'? It's in the list! Here's another suggestion: '{random_song}'")
    else:
        print("Sorry, your song is not hot enough.")

You can type 'exit' at any time to quit.


Please enter a song:  juno


Did you mean 'Juno'? It's in the list! Here's another suggestion: 'Thought I Was Dead'


Please enter a song:  I was dead


Sorry, your song is not hot enough.


Please enter a song:  ecit


Sorry, your song is not hot enough.


Please enter a song:  exit


Goodbye!


<h2> Dataframe Audio_features </h2>

In [ ]:
# Using the Spotipy library, create a DataFrame storing the audio_features of at least 1000 songs.

In [68]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))